In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from IPython.display import clear_output
from tqdm import tqdm

In [3]:
from typing import Dict, List, Tuple
from buffer_pool_baseline.timer import Time
from buffer_pool_baseline.cache import Cache
from buffer_pool_baseline.strategy import EvictionStrategy
from buffer_pool_baseline.environment import Query

In [4]:
%matplotlib inline

In [5]:
def setup():
    
    t = Time(0)
    
    c = Cache(cache_size, t, equate_id_to_value=True)
    
    start = random.randint(0, 20)
    end = start + 5
    
    start_2 = start + random.randint(0, 5)
    end_2 = start_2 + 5
    
    loop_size = random.randint(1, 3)
    
    select = Query(query_type = "select", time=t, parameters={"start": start, "end": end})
    env = Query(query_type="join", time=t, parameters={"start_table_1": start, "end_table_1": end, 
                                                       "start_table_2": start_2, "end_table_2": end_2})
    env.set_query_cache(c)
    
    c.add_element(random.randint(0, 30))
    c.add_element(random.randint(0, 30))
    c.add_element(random.randint(0, 30))
    c.add_element(random.randint(0, 30))
    return env

In [6]:
def multisetup():
    
    t = Time(0)
    
    c = Cache(cache_size, t, equate_id_to_value=True)
    
    start = random.randint(0, 20)
    end = start + 5
    
    join_start = random.randint(end+10, end+15)
    join_end = join_start + 5
    
    start_2 = join_start + random.randint(5, 10)
    end_2 = start_2 + 5
    
    loop_size = random.randint(1, 3)
    
#     Change queries as per use case here! Name them select and join_query if you are lazy.

    select = Query(query_type = "sequential", time=t, parameters={"start": start, "end": end, "loop_size": loop_size})
#     join_query = Query(query_type = "sequential", time=t, parameters={"start": start_2, "end": end_2, "loop_size": loop_size})
    
#     select = Query(query_type = "join", time=t, parameters={"start_table_1": join_start, "end_table_1": join_end, 
#                                                             "start_table_2": start_2, "end_table_2": end_2})
    
#     select = Query(query_type="join", time=t, parameters={"start_table_1": join_start, "end_table_1": join_end, 
#                                                        "start_table_2": start_2, "end_table_2": end_2})
    
    join_query = Query(query_type="sequential", time=t, parameters={"start": join_start, "end": join_end, "loop_size": loop_size})
    
#     c.add_element(random.randint(100, 300))
#     c.add_element(random.randint(100, 300))
#     c.add_element(random.randint(100, 300))
#     c.add_element(random.randint(100, 300))
    select.set_query_cache(c)
    join_query.set_query_cache(c)
    
    return select, join_query

In [7]:
def get_query(query_type, table_size, loop_size):
    t = Time()
    if query_type == "sequential":
        return Query(query_type="sequential", time=t, parameters={"start": 0, "end": table_size, "loop_size": loop_size})
    
    if query_type == "select":
        return Query(query_type="select", time=t, parameters={"start": 0, "end": table_size})
    
    if query_type == "join":
        return Query(query_type="join", time=t, parameters={"start_table_1": 0, "end_table_1": 10, 
                                                            "start_table_2": 0, "end_table_2": 10})
    

In [8]:
def get_time_steps_per_query(table_size, cache_size, loop_size):
    time_steps = {}
    
    query_types = ["sequential", "select", "join"]
    
    for query in query_types:
        t = Time(0)
        env = get_query(query, table_size, loop_size)
        c = Cache(cache_size, t, equate_id_to_value=True)
        env.set_query_cache(c)

        while not env.is_done():
            env.step("mru")

        time_steps[query] = env.time.now()
    
    return time_steps

In [9]:
def encode_queries(query_type, time_step, max_time_steps):
    return max_time_steps[query_type] + time_step

In [10]:
max_time_steps = get_time_steps_per_query(10, 4, 5)

In [11]:
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

epsilon_greedy = True
alpha_decay = True

In [12]:
q_values_cumulative = []
old_q = np.zeros([1000, 4])
q_table = np.zeros([1000, 4])
cache_size = 4
number_of_runs = 1000000

In [13]:
epsilon_div = number_of_runs / 6
alpha_div = number_of_runs / 6

In [14]:
def evaluate_policy(query, q_table):
    cache_map_policy = []
    
    start_index = max_time_steps[query.query_type]
    
    reward = 0
    
    for d in q_table[start_index:]:
        action = ["mru", "lru", "lfu", "fifo"][np.argmax(d)]
        cache_map_policy.append({"action": action, 
                                 "cache": query.cache.cache_map.copy().keys(),
                                "which_element": query.parameters.copy()})
        query.step(action)
        
        if query.is_done():
            break
    
    hits, misses = query.step()
    reward = hits -  misses
    return hits, misses
    

In [15]:
def get_baseline(query, action = "lru"):
    cache_map_baseline = []
    
    while not query.is_done():
        query.step(action)
        cache_map_baseline.append({"cache": query.cache.cache_map.copy().keys(), 
                                   "which_element": query.parameters.copy()})
        
    hits, misses = query.step()
    return hits, misses

In [16]:
def compare_query_reward(env):
    params_baseline = env.parameters.copy()
    params_random = env.parameters.copy()
    params_policy = env.parameters.copy()
    query_type = env.query_type
    
    t = Time(0)
    query_baseline = Query(query_type=query_type, parameters=params_baseline, time=t)
    c = Cache(cache_size, time=t, equate_id_to_value=True)
    query_baseline.set_query_cache(c)
    
    query_baseline_random = Query(query_type=query_type, parameters=params_random, time=t)
    c2 = Cache(cache_size, time=t, equate_id_to_value=True)
    query_baseline_random.set_query_cache(c2)
    
    query_policy = Query(query_type=query_type, parameters=params_policy, time=t)
    c3 = Cache(cache_size, time=t, equate_id_to_value=True)
    query_policy.set_query_cache(c3)
    
    hits_policy, misses_policy = evaluate_policy(query_policy, q_table)
    hits_baseline, misses_baseline = get_baseline(query_baseline)
    hits_random, misses_random = get_baseline(query_baseline_random, "random")
    
    return {"hits_random" : hits_random, "misses_random": misses_random, 
            "hits_baseline": hits_baseline, "misses_baseline": misses_baseline,
           "hits_policy": hits_policy, "misses_policy": misses_policy, "tm": query_baseline.time.now()}

In [17]:
encoded_state = []

def run_multiple():
    global epsilon, alpha
    
    cum_reward_plot = []
    q_values_cumulative = []
    reward_policy = []
    
    for i in tqdm(range(1, number_of_runs+1)):

        q1, q2 = multisetup()
        state = encode_queries(q1.query_type, q1.time.now(), max_time_steps)

        penalties, reward, = 0, 0
        done = False
        previous_hit, previous_miss = 0, 0
        cum_reward = 0
        tm = 0
        
        old_q = q_table.copy()
        
        while (not q1.is_done() or not q2.is_done()):
            if q1.is_done():
                env = q2
            else:
                env = q1
            
            if epsilon_greedy:
                epsilon = 0.1/pow(10, round(i/epsilon_div))
            
            if random.uniform(0, 1) < epsilon:
                action = random.choice(env.actions) # Explore action space
            else:
                _action_ = np.argmax(q_table[state]) # Exploit learned values
                action = ["mru", "lru", "lfu", "fifo"][_action_]

            hits, miss = env.step(action)
            next_state = encode_queries(env.query_type, env.time.now(), max_time_steps)
            encoded_state.append(next_state)
        
            reward = hits - previous_hit
            penalties = miss - previous_miss

            previous_hit = hits
            previous_miss = miss

            _action_ = ["mru", "lru", "lfu", "fifo"].index(action)

            old_value = q_table[state, _action_]
            next_max = np.max(q_table[next_state])

            r = reward - penalties
            
            if alpha_decay:
                alpha = 0.01/pow(10, round(i/alpha_div))
                
            new_value = (1 - alpha) * old_value + alpha * (r + gamma * next_max)
            
            q_table[state, _action_] = new_value
            state = next_state
            cum_reward += r
            tm += 1
            
        q_values_cumulative.append({"q_values_cumulative": sum(sum(abs(old_q - q_table)))})
        
        cum_reward_plot.append({"reward": cum_reward, "epoch": i, "time": tm})
        
        if i % 100 == 0:
            total_hits = 0
            total_misses = 0

            for i in range(100):
                q1, q2 = multisetup()
                r1 = compare_query_reward(q1) 
                r2 = compare_query_reward(q2)

                total_r = {}
                for x, y in r1.items():
                    total_r[x] = y

                for x, y in r2.items():
                    total_r[x] += y
                    
                tm = q1.time.now() + q2.time.now()
                total_hits += total_r["hits_policy"] / total_r["tm"]
                total_misses += total_r["misses_policy"] / total_r["tm"]
            
            reward_policy.append({"Total Hits": total_hits, "Total Misses": total_misses})
            
    return cum_reward_plot, q_values_cumulative, reward_policy
    print("Training finished.\n")


In [ ]:
cum_reward_plot, q_values_cumulative, reward = run_multiple()

  3%|██▏                                                                      | 30287/1000000 [01:44<51:31, 313.68it/s]

In [ ]:
def compare_reward(query_type):
    
    env = setup()
    params_baseline = env.parameters.copy()
    params_random = env.parameters.copy()
    params_policy = env.parameters.copy()
    
    t = Time(0)
    query_baseline = Query(query_type=query_type, parameters=params_baseline, time=t)
    c = Cache(cache_size, time=t, equate_id_to_value=True)
    query_baseline.set_query_cache(c)
    
    query_baseline_random = Query(query_type=query_type, parameters=params_random, time=t)
    c2 = Cache(cache_size, time=t, equate_id_to_value=True)
    query_baseline_random.set_query_cache(c2)
    
    query_policy = Query(query_type=query_type, parameters=params_policy, time=t)
    c3 = Cache(cache_size, time=t, equate_id_to_value=True)
    query_policy.set_query_cache(c3)
    
    hits_policy, misses_policy = evaluate_policy(query_policy, q_table)
    hits_baseline, misses_baseline = get_baseline(query_baseline)
    hits_random, misses_random = get_baseline(query_baseline_random, "random")
    
    return {"hits_random" : hits_random, "misses_random": misses_random, 
            "hits_baseline": hits_baseline, "misses_baseline": misses_baseline,
           "hits_policy": hits_policy, "misses_policy": misses_policy, "tm": query_baseline.time.now()}

In [ ]:
df_metrics = pd.DataFrame(reward)

In [ ]:
(df_metrics["Total Hits"]).plot()
plt.title("Number of hits for multiple sequential + sequential queries for empty cache and 4 actions")
plt.xlabel("Validation Run")
plt.ylabel("Hits")

In [ ]:
# q1, q2 = multisetup()

# h1, miss1 = evaluate_policy(q1, q_table)
# h2, miss2 = evaluate_policy(q2, q_table)

In [ ]:
# h1, miss1, h2, miss2

In [ ]:
# q1, q2 = multisetup()
# get_baseline(q1), get_baseline(q2)

In [ ]:
df_q_value = pd.DataFrame(q_values_cumulative)

In [ ]:
# df_q_value.min(), df_q_value.max()

In [ ]:
df = pd.DataFrame(cum_reward_plot)

In [ ]:
df["r"] = df["reward"] / df["time"]

In [ ]:
df.reward.max()

In [ ]:
df["rolling_mean"] = df["r"].rolling(200).mean()

In [ ]:
df_q_value = pd.DataFrame(q_values_cumulative)
df_q_value.head()
df_q_value["rolling"] = df_q_value.rolling(1000).mean()

In [ ]:
df_q_value.plot(figsize=(20,5))

In [ ]:
df.plot(y="rolling_mean", x="epoch", figsize=(20, 10))

In [ ]:
max_time_steps

In [ ]:
# Heatmap
actions = ["mru", "lru", "lfu", "fifo"]
fig, ax = plt.subplots()
im = ax.imshow(q_table[66:111])

# We want to show all ticks...
ax.set_xticks(np.arange(len(actions)))

ax.set_xticklabels(actions)


# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(q_table[66:111])):
    for j in range(len(actions)):
        #print(q_table[66+i, j])
        text = ax.text(j, i, round(q_table[66+i, j],4),
                       ha="center", va="center", color="w")
fig.set_figheight(40)
fig.set_figwidth(20)
ax.set_title("Actions chosen by our model for each time step")
plt.show()

In [ ]:
# def run_single():
#     global epsilon
    
#     cum_reward_plot = []
#     encoded_state = []

#     for i in tqdm(range(1, number_of_runs)):

#         env = setup()
#         state = encode_queries(env.query_type, env.time.now(), max_time_steps)

#         penalties, reward, = 0, 0
#         done = False
#         previous_hit, previous_miss = 0, 0
#         cum_reward = 0
#         tm = 0

#         while not done:

#             if random.uniform(0, 1) < epsilon:
#                 action = random.choice(env.actions) # Explore action space
#             else:
#                 _action_ = np.argmax(q_table[state]) # Exploit learned values
#                 action = "mru" if _action_ == 0 else "lru"

#             hits, miss = env.step(action)
#             next_state = encode_queries(env.query_type, env.time.now(), max_time_steps)
#             encoded_state.append(next_state)

#             reward = hits - previous_hit
#             penalties = miss - previous_miss

#             previous_hit = hits
#             previous_miss = miss

#             done = env.done

#             _action_ = 0 if action == "mru" else 1

#             old_value = q_table[state, _action_]
#             next_max = np.max(q_table[next_state])

#             r = reward - penalties
#             new_value = (1 - alpha) * old_value + alpha * (r + gamma * next_max)
#             q_table[state, _action_] = new_value
#             state = next_state
#             cum_reward += r
#             tm += 1

#         old_q = q_table.copy()

#         cum_reward_plot.append({"reward": cum_reward, "epoch": i, "time": tm})

#     print("Training finished.\n")